In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import linregress
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten, concatenate, Conv2D
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
gesture_features = pd.DataFrame.from_csv("../../data/gesture_feature_df.csv")
train_data = pd.DataFrame.from_csv("../../data/database/special/train_data.csv")
gesture_features = gesture_features[(gesture_features.TagName == 'Tag0') | (gesture_features.TagName == 'Tag6')]
train_data = train_data[(train_data.TagName == 'Tag0') | (train_data.TagName == 'Tag6')]

In [3]:
# all_data = train_data.groupby(['TagName','tester_id'])
# all_keys = all_data.groups.keys()
# all_keys = list(all_keys)
# np.random.shuffle(all_keys)
# test_keys = all_keys[:int(len(all_keys)/4)]
# test_gesture = []
# train_gesture = []

# for i in range(len(gesture_features)):
#     cur_key = (gesture_features.iloc[i]['TagName'],gesture_features.iloc[i]['tester_id'])
#     if cur_key in test_keys:
#         test_gesture.append(gesture_features.iloc[i].values)
#     else:
#         train_gesture.append(gesture_features.iloc[i].values)
        
# test_gesture = pd.DataFrame(test_gesture,columns=gesture_features.columns)
# train_gesture = pd.DataFrame(train_gesture,columns=gesture_features.columns)

all_data = train_data.groupby(['TagName','tester_id'])
all_keys = all_data.groups.keys()
X = []
y = []

for i in all_keys:
    y.append(i[0])
    X.append(gesture_features[((gesture_features.TagName == i[0]) & (gesture_features.tester_id == i[1]))]['d_change'].values[0])


X = np.array(X).reshape(-1,1)
y = np.array(y)

In [4]:
from sklearn.model_selection import cross_val_score, cross_val_predict

In [17]:
# clf = RandomForestClassifier(n_estimators=30,criterion='gini',max_depth=10)
clf = DecisionTreeClassifier(max_depth=2)
np.mean(cross_val_score(clf,X,y,cv=10))

0.82499999999999996

In [21]:
# res = cross_val_predict(clf,X,y,cv=10)
res = clf.predict(X)

In [22]:
print(classification_report(y, res))
print(confusion_matrix(y, res)) 

             precision    recall  f1-score   support

       Tag0       0.93      0.78      0.85        83
       Tag6       0.81      0.94      0.87        83

avg / total       0.87      0.86      0.86       166

[[65 18]
 [ 5 78]]


In [20]:
clf.fit(X,y)
from sklearn.externals import joblib
joblib.dump(clf, './binary_model.pkl')

['./binary_model.pkl']

In [63]:
(66+64)/166

0.7831325301204819

In [65]:
len(gesture_features)

208

In [66]:
208-166

42

In [67]:
166*5

830

In [68]:
1040-830

210